# Data Cleaning

## Team : 14 (Vidisha, Yijin, Yvette)


In [69]:
import pandas as pd
data_20 = pd.read_csv('adult20csv/adult20.csv') # Reading Data for 2020
data_20

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,RJWKCLSOFT_A,RJWCLSNOSD_A,RJWRKCLSSD_A,RECJOBSD_A,...,PHSTAT_A,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,WTFA_A,HHX,POVRATTC_A
0,3,14,5,0,100000,0,NaN,NaN,NaN,NaN,...,2,NaN,NaN,1,1,11,10,4526.109,H066706,6.47
1,3,11,4,0,75000,0,NaN,NaN,NaN,NaN,...,2,NaN,NaN,1,1,8,10,12809.039,H034928,3.64
2,3,14,4,0,90000,0,NaN,NaN,NaN,NaN,...,3,NaN,NaN,1,1,8,10,10322.534,H018289,6.76
3,3,11,3,0,65000,0,NaN,NaN,NaN,NaN,...,1,NaN,NaN,1,1,3,10,7743.375,H006876,3.79
4,3,8,1,0,25762,2,NaN,NaN,NaN,NaN,...,3,NaN,NaN,1,1,6,10,4144.724,H028842,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31563,4,13,4,0,79000,0,NaN,NaN,NaN,NaN,...,3,NaN,NaN,1,1,2,10,2857.585,H065697,4.61
31564,4,11,3,0,60000,0,NaN,NaN,NaN,NaN,...,3,NaN,NaN,1,1,2,10,2994.763,H061937,3.50
31565,4,8,1,0,27500,0,NaN,NaN,NaN,NaN,...,2,NaN,NaN,1,1,2,10,1328.907,H005331,2.24
31566,4,8,3,0,61880,0,NaN,NaN,NaN,NaN,...,2,NaN,NaN,1,1,2,10,3481.003,H047025,2.38


### Step 1: Dealing with missing values

Since we are using survey data, there are bound to be several missing values in various attributes due to non-response. In order to deal with such missing observations, we consider only those columns in the data set with number of missing values less than 10% of the total observations. 

We consider 10% as a reasonable threshold for a data set with 31,586 observations. For columns with number of missing vales less than 10% of the total observations, we drop the missing values as these are a small proportion of the total observations.

In [70]:
prop_missing = data_20.isna().sum()/ len(data_20) # Proportion of missing values in every attribute
keep = list(prop_missing[prop_missing < 0.1].index) # Attributes with less than 10% missing observations
data_20 = data_20[keep] 

### Remaining Missing Values


In [71]:
percent_missing = data_20.isnull().sum() * 100 / len(data_20) #quantifying proportion of missing observations
missing_values_df = pd.DataFrame({'column_name': data_20.columns,
                                 'percent_missing': percent_missing}) #displaying variable name and proportion of missing observations
print(missing_values_df)

data_20 = data_20.dropna() #Dropping missing values in remaining variables

print(data_20)

           column_name  percent_missing
URBRRL          URBRRL              0.0
RATCAT_A      RATCAT_A              0.0
INCGRP_A      INCGRP_A              0.0
INCTCFLG_A  INCTCFLG_A              0.0
FAMINCTC_A  FAMINCTC_A              0.0
...                ...              ...
INTV_MON      INTV_MON              0.0
RECTYPE        RECTYPE              0.0
WTFA_A          WTFA_A              0.0
HHX                HHX              0.0
POVRATTC_A  POVRATTC_A              0.0

[212 rows x 2 columns]
       URBRRL  RATCAT_A  INCGRP_A  INCTCFLG_A  FAMINCTC_A  IMPINCFLG_A  PPSU  \
0           3        14         5           0      100000            0     2   
3           3        11         3           0       65000            0     2   
5           3         9         2           0       36000            0     2   
6           3         3         1           0       30105            2     2   
7           2         2         1           0        9000            1     2   
...       ...   

### Step 2: Removing irrelevant columns

Our original dataset included over 600 variables. In this step, we manually selected variables using the dataset codebook (https://ftp.cdc.gov/pub/Health_Statistics/NCHS/Dataset_Documentation/NHIS/2020/adult-codebook.pdf) and created an excel sheet (relevant_variables.xlsx) with all the variables relevant for our study topic. We decided to take this approach given the large number of variables that were similar. For example, there were several variables related to food insecurity (e.g., Couldn't afford to eat balanced meals; food didn't last; worry food would run out;receive food stamps, past 12m). For each group of similar variables, we chose the one captured the most information.

We also chose to manually select our remaining variables since there were many features irrelevant to our research questions. We chose to manually eliminate these variables as well.

In [72]:
relevant_variables = pd.read_excel('relevant_variables.xlsx')
data_20 = data_20[list(relevant_variables["Variable Names"])]

data_20

,FDSCAT4_A,SOCSCLPAR_A,VIGFREQW_A,HIKIND01_A,PAYBLL12M_A,PAYWORRY_A,DEMENEV_A,DEPMED_A,DEPFREQ_A,ANGEV_A,...,MHTHRPY_A,SMKCIGST_A,SMKECIGST_A,DRKSTAT_A,MODTPR_A,MODFREQW_A,LEGMSTAT_A,CITZNSTP_A,INCGRP_A,FDSCAT3_A
0,1,1,94,2,2,3,2,2,5,2,...,2,4,3,7,0,94,3,1,5,1
3,1,1,5,2,2,2,2,2,5,2,...,2,4,3,6,2,5,4,1,3,1
5,2,1,94,1,2,2,2,1,1,2,...,2,4,3,5,2,7,5,1,2,1
6,2,1,94,1,2,2,2,2,3,2,...,2,4,3,2,2,1,3,1,1,1
7,1,1,94,2,1,3,2,2,5,2,...,2,4,3,2,0,94,5,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31558,1,1,0,1,2,2,2,2,4,2,...,2,3,1,3,3,1,3,1,4,1
31560,1,1,2,1,2,2,2,1,2,2,...,2,3,3,3,2,5,2,1,1,1
31564,1,1,94,1,2,3,2,2,4,2,...,2,4,3,5,2,3,3,1,3,1
31565,1,1,94,2,2,3,2,2,5,2,...,2,3,3,8,0,94,5,1,1,1


### 

In [73]:
data_19 = pd.read_csv('adult19csv/adult19.csv') # Reading Data for 2019
data_19

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_QRT,RECTYPE,WTFA_A,WTIA_A,HHX,POVRATTC_A
0,4,9,3,0,60000,2,2,122,3,2,...,NaN,NaN,1,1,1,10,13177.008,7601.336,H048109,2.96
1,4,9,3,0,50000,0,2,122,2,1,...,NaN,NaN,1,1,1,10,6140.552,3344.434,H027044,2.97
2,4,12,3,0,65000,1,2,122,2,1,...,NaN,NaN,1,1,1,10,9191.061,6949.498,H058855,4.28
3,4,14,5,0,120000,0,2,122,1,8,...,NaN,NaN,1,1,1,10,7900.035,6446.327,H031993,7.13
4,1,4,1,0,30000,0,2,115,2,1,...,NaN,NaN,1,1,1,10,10875.772,8646.586,H007122,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31992,4,14,5,0,116204,0,100,114,2,1,...,NaN,NaN,1,1,4,10,2679.016,1965.416,H046022,7.65
31993,4,8,3,0,68000,0,100,114,2,1,...,NaN,NaN,1,1,4,10,6726.495,3920.208,H046232,2.02
31994,4,13,3,0,60000,0,100,114,2,1,...,NaN,NaN,1,1,4,10,1246.306,864.743,H043765,4.98
31995,4,14,5,0,101000,0,100,114,1,1,...,NaN,NaN,1,1,4,10,3427.198,2595.209,H017695,6.00


In [74]:
prop_missing = data_19.isna().sum()/ len(data_19) # Proportion of missing values in every attribute
keep = list(prop_missing[prop_missing < 0.1].index) # Attributes with less than 10% missing observations
data_19 = data_19[keep] 
data_19 = data_19.dropna() #Dropping missing values in remaining variables

data_19

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,HYPEV_A,PHSTAT_A,AVAIL_A,HHSTAT_A,INTV_QRT,RECTYPE,WTFA_A,WTIA_A,HHX,POVRATTC_A
0,4,9,3,0,60000,2,2,122,3,2,...,2,3,1,1,1,10,13177.008,7601.336,H048109,2.96
1,4,9,3,0,50000,0,2,122,2,1,...,2,1,1,1,1,10,6140.552,3344.434,H027044,2.97
2,4,12,3,0,65000,1,2,122,2,1,...,1,3,1,1,1,10,9191.061,6949.498,H058855,4.28
3,4,14,5,0,120000,0,2,122,1,8,...,1,2,1,1,1,10,7900.035,6446.327,H031993,7.13
4,1,4,1,0,30000,0,2,115,2,1,...,1,4,1,1,1,10,10875.772,8646.586,H007122,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31991,4,10,3,0,68000,0,100,114,2,1,...,2,2,1,1,4,10,3551.733,1501.538,H033834,3.36
31992,4,14,5,0,116204,0,100,114,2,1,...,1,3,1,1,4,10,2679.016,1965.416,H046022,7.65
31994,4,13,3,0,60000,0,100,114,2,1,...,1,2,1,1,4,10,1246.306,864.743,H043765,4.98
31995,4,14,5,0,101000,0,100,114,1,1,...,2,2,1,1,4,10,3427.198,2595.209,H017695,6.00


In [81]:
#data_19 = data_19[list(relevant_variables["Variable Names"])]

#data_19

### Step 3: Assessing Categorical Variables

Now that we have our final features, we assess how many categories each categorical variables has.

In [76]:
for col in list(data_20):
    print('variable:', col) 
    print(data_20[col].value_counts()) #printing number of observations for each category

variable: FDSCAT4_A
1    19892
2      913
3      621
4      418
8       46
Name: FDSCAT4_A, dtype: int64
variable: SOCSCLPAR_A
1    20493
2      982
3      286
4      121
9        7
7        1
Name: SOCSCLPAR_A, dtype: int64
variable: VIGFREQW_A
94    13591
1      1797
2      1527
3      1403
0      1244
7       650
4       616
5       543
6       153
96      128
99       79
95       44
14       38
21       31
28       17
97       14
98        4
10        4
20        2
12        2
22        1
8         1
9         1
Name: VIGFREQW_A, dtype: int64
variable: HIKIND01_A
1    14245
2     7628
7        9
9        8
Name: HIKIND01_A, dtype: int64
variable: PAYBLL12M_A
2    19996
1     1863
9       21
7        9
8        1
Name: PAYBLL12M_A, dtype: int64
variable: PAYWORRY_A
3    13174
2     6589
1     2089
9       31
7        7
Name: PAYWORRY_A, dtype: int64
variable: DEMENEV_A
2    21748
1      130
7       10
9        2
Name: DEMENEV_A, dtype: int64
variable: DEPMED_A
2    19469
1     2384


### Step 4: Transforming Categorical Variables

In our dataset, categorical variables are encoding using integers. Below, we convert the categorical features that are int64 to the categorical datatype. This allows us to transform the categorical variables into dummy variables.

In [82]:
print(data_20.dtypes) #check which variables are categorical

for col in list(data_20): #converting int64 to categorical
    if str(data_20[col].dtypes) == 'int64':
        data_20[col] = pd.Categorical(data_20[col])
    
data_20 = pd.get_dummies(data_20, drop_first = True) #convert categorical into dummy variables

print(data_20)

data_20.to_csv("data_2020_cleaned.csv")

POVRATTC_A     float64
STEPS_A        float64
WELLNESS_A     float64
WLK100_A       float64
WLK13M_A       float64
                ...   
INCGRP_A_4       uint8
INCGRP_A_5       uint8
FDSCAT3_A_2      uint8
FDSCAT3_A_3      uint8
FDSCAT3_A_8      uint8
Length: 3592, dtype: object
       POVRATTC_A  STEPS_A  WELLNESS_A  WLK100_A  WLK13M_A  INCOTHR_A  \
0            6.47      1.0         1.0       1.0       1.0        2.0   
3            3.79      1.0         1.0       1.0       1.0        2.0   
5            2.93      1.0         1.0       1.0       1.0        2.0   
6            0.98      1.0         2.0       1.0       1.0        2.0   
7            0.73      3.0         1.0       3.0       3.0        2.0   
...           ...      ...         ...       ...       ...        ...   
31558        4.96      1.0         1.0       1.0       1.0        2.0   
31560        2.55      1.0         1.0       1.0       1.0        2.0   
31564        3.50      2.0         1.0       1.0       1.0    

### Step 5: Visualization

### Step 6: Preliminary Test